# Bibliotecas

In [1]:
import pandas as pd
from pathlib import Path
import os

# Tratar bases

In [2]:
# Diretório base onde o arquivo CSV está localizado
base_diretório = Path(os.getcwd())
diretório_princapl = base_diretório.parent
caminho = diretório_princapl / 'Base'

# Definir o caminho completo da base não tratada do ibovespa
arquivo_ibovespa = caminho / 'Dados Históricos - Ibovespa.csv'

In [3]:
def tratar_dados(caminho_csv):
    # Força leitura como string para evitar perda de zeros
    acao = pd.read_csv(caminho_csv, dtype=str)

    # Converter a coluna de data
    acao['Data'] = pd.to_datetime(acao['Data'], format='%d.%m.%Y')

    # Corrigir colunas numéricas
    colunas_dinheiro = ['Último', 'Abertura', 'Máxima', 'Mínima']
    for coluna in colunas_dinheiro:
        acao[coluna] = (
            acao[coluna]
            .str.replace('.', '', regex=False)  # Remove separador de milhar
            .str.replace(',', '', regex=False)  # Remove separador decimal
            .astype(float)
        )

    # Corrigir variação percentual
    acao['Var%'] = (
        acao['Var%']
        .str.replace('%', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 100
    )
    acao['Var%'] = acao['Var%'].round(4)
    acao = acao.rename(columns={'Var%': 'Variacao (%)'})

    # Remover a coluna de volume
    acao = acao.drop(columns=['Vol.'])

    # Ordenar por data
    acao = acao.sort_values(by='Data')

    return acao

# Aplicar função e salvar resultado
df_ibovespa = tratar_dados(arquivo_ibovespa)
df_ibovespa.to_csv("Base Ibovespa Tratada.csv", index=False)


In [4]:
df_ibovespa

,Data,Último,Abertura,Máxima,Mínima,Variacao (%)
0,2015-01-05,47517.0,48512.0,48512.0,47264.0,-0.0205
1,2015-01-06,48001.0,47517.0,48061.0,47338.0,0.0102
2,2015-01-07,49463.0,48006.0,49882.0,48006.0,0.0305
3,2015-01-08,49943.0,49463.0,50261.0,49017.0,0.0097
4,2015-01-09,48840.0,49955.0,49955.0,48501.0,-0.0221
...,...,...,...,...,...,...
2577,2025-05-28,138888.0,139541.0,139547.0,138580.0,-0.0047
2578,2025-05-29,138534.0,138869.0,139108.0,137993.0,-0.0025
2579,2025-05-30,137027.0,138546.0,138637.0,136726.0,-0.0109
2580,2025-06-02,136787.0,137026.0,138471.0,136483.0,-0.0018


In [5]:
df_ibovespa.dtypes

Data            datetime64[ns]
Último                 float64
Abertura               float64
Máxima                 float64
Mínima                 float64
Variacao (%)           float64
dtype: object